In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import tensorflow as tf

In [2]:
plt.rcParams['figure.figsize'] = [12., 8.]
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14 
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 14

# Problem

Let's assume an isotropic light source sending out a fixed amount of photons $N_{src}$ at time $t_{src}$ at position $x_{src}$ that is located $b=1 m$ off from a line of detectors.
The amount of photons expected at each detector is $N_{src}/r^2$ (and the observed $N_i$ drawn according to Poisson) where $r$ is the distance to the detector. The arrival time is distributed to $\mathcal{N}(\mu=t_{rsc} + r/c, \sigma=1)$.

The extended Likelihood formulation then gives the likelihood for photons observed per detector:

$L_i = \prod_{hit = 0}^{N_i} \mathcal{N}(\mu=t_{rsc} + r_i/c - t_{hit}, \sigma=1)\cdot e^{-N_{src}/r_i^2} \cdot (N_{src}/r_i^2)^{N_i}$

The total likelihood is the the product over all detectors.

distances are in meters, time in ns (not that it matters anyway)

In [3]:
time_dist = lambda t: stats.norm(loc=t, scale=1)
c = 0.3 # m/ns

In [13]:
# 3 detectors

detector_xs = np.linspace(-10,10,3)

In [14]:
def generate_event(x_src, t_src=0, N_src=10, b=1):
    '''
    generates event
    
    Parameters:
    
    x_rsc : float
        Source position
    t_src : float
        Source time
    N_src : int
        Amount of photons sent out
    b : float
        perpendicaulr distance off of sensor line
        
    Returns:
    
    Ns : list
        observed number of photons per detector
    Ns_sensor_idx : list
        according index of sensor
    ts : list
        observed photon times
    ts_sensor_idx : list
        according index of sensor
    '''
    Ns = []
    Ns_sensor_idx = []
    ts = []
    ts_sensor_idx = []
    for i, x in enumerate(detector_xs):
        r2 = (x_src - x)**2 + b**2
        N_exp = N_src/r2
        N_obs = stats.poisson(mu=N_exp).rvs()
        Ns.append(N_obs)
        Ns_sensor_idx.append(i)
        if N_obs > 0:
            t = time_dist(t_src + np.sqrt(r2)/c).rvs(size=N_obs)
            ts.extend(t)
            ts_sensor_idx.extend([i]*N_obs)
    return Ns, Ns_sensor_idx, ts, ts_sensor_idx

In [40]:
events = []
for i in range(2):
    events.append(generate_event(-5))

In [41]:
events

[([0, 2, 1],
  [0, 1, 2],
  [15.34907884312193, 16.475652350001095, 50.462725461967295],
  [1, 1, 2]),
 ([0, 0, 0], [0, 1, 2], [], [])]

In [17]:
detector_xs

array([-10.,   0.,  10.])